# MFCC-Euclidean baseline

In [1]:
import numpy as np
import pickle
import librosa
from scipy.spatial.distance import cdist
import python_speech_features
import numba
from matplotlib import pyplot as plt
from matplotlib import gridspec
import time
import os
import pandas as pd
import glob
from tqdm import tqdm
from multiprocessing import Pool
from multiprocessing import cpu_count
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import roc_curve, auc
from offset import find_offset as offset_hps
import random
from eer_utils import *

In [2]:
mfcc_DIR = './daps-mp3/test/mfccs/'
#HP_DIR = './daps-mp3/train/hashprints/'

queries = ['queries/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'queries/'))]
tamp_025 = ['tampered0.25/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'tampered0.25/'))]
tamp_05 = ['tampered0.5/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'tampered0.5/'))]
tamp_1 = ['tampered1/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'tampered1/'))]
tamp_2 = ['tampered2/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'tampered2/'))]
tamp_4 = ['tampered4/' + file[:-4] for file in sorted(os.listdir(mfcc_DIR + 'tampered4/'))]

In [3]:
#ref_hp_dict = {file[:-10]: np.load(HP_DIR + 'refs/'+file) for file in sorted(os.listdir(HP_DIR + 'refs/'))}
ref_mfcc_dict = {file[:-10]: np.load(mfcc_DIR + 'refs/'+file) for file in sorted(os.listdir(mfcc_DIR + 'refs/'))}

In [4]:
all_queries = queries + tamp_025 + tamp_05 + tamp_1 + tamp_2 + tamp_4

In [5]:
def calculate_tamper_score(query):
    startTime = time.time()
    
    query_type, query_name = query.split('/')

    if query_type == "queries":
        tamper_type = "NONE"
        tamper_len = 0.
    else:
        tamper_type = query_name[:3].upper()
        tamper_len = float(query_type[len('tampered'):])
    
    _, query_no, speaker, script, _ = query_name.split('_')
    _, bitrate = query_name.split('-')
    ref_name = f'{speaker}_{script}'
    
    # load query mfcc
    query_mfcc = np.load(mfcc_DIR + query + '.npy')
    
    # load ref mfcc
    ref_mfcc = ref_mfcc_dict[ref_name]
    
    # threshold delta delta and find offset
    query_mhps = np.dot(query_mfcc[:,13:] > 0,np.power(2,np.arange(26))[::-1]).tolist()
    ref_mhps = np.dot(ref_mfcc[:,13:] > 0,np.power(2,np.arange(26))[::-1]).tolist()
    
    offset = offset_hps(query_mhps, ref_mhps)
    ref_mfcc = ref_mfcc[offset:offset+len(query_mfcc)]
    
    tamper_score = np.sqrt(np.square(abs(query_mfcc - ref_mfcc)).sum(axis=1)).mean()

    
    return tamper_type, tamper_len, bitrate, ref_name, query_no, tamper_score

In [6]:
for i in np.random.randint(0, len(queries), 10):
    print(calculate_tamper_score(queries[i]))

('NONE', 0.0, '256k', 'f10_script3', '9', 16.820102239578258)
('NONE', 0.0, '128k', 'f7_script4', '9', 13.726691248067652)
('NONE', 0.0, '256k', 'f6_script2', '6', 14.573975353535104)
('NONE', 0.0, '128k', 'f8_script3', '0', 17.396563890313153)
('NONE', 0.0, '256k', 'f10_script5', '8', 16.532210279557)
('NONE', 0.0, '64k', 'f6_script4', '5', 14.74676736159454)
('NONE', 0.0, '256k', 'f6_script3', '4', 14.152366870367258)
('NONE', 0.0, '256k', 'm10_script3', '4', 15.793534201849821)
('NONE', 0.0, '64k', 'm9_script2', '5', 13.940201919525101)
('NONE', 0.0, '64k', 'm8_script4', '7', 15.6983304663547)


In [7]:
for i in np.random.randint(3000, 4500, 10):
    print(calculate_tamper_score(tamp_4[i]))

('REP', 4.0, '128k', 'f8_script4', '0', 39.603303435407675)
('REP', 4.0, '256k', 'f9_script2', '1', 41.883873592061825)
('REP', 4.0, '64k', 'm9_script5', '2', 37.03723005155455)
('REP', 4.0, '128k', 'm6_script2', '1', 40.99096867794243)
('REP', 4.0, '64k', 'm7_script3', '3', 44.573093434086076)
('REP', 4.0, '128k', 'm7_script5', '4', 39.57301161961814)
('REP', 4.0, '128k', 'm10_script1', '5', 41.028615028691554)
('REP', 4.0, '64k', 'f9_script5', '8', 40.81482103263883)
('REP', 4.0, '256k', 'm7_script4', '5', 40.07505359379813)
('REP', 4.0, '64k', 'm6_script3', '1', 40.55032964717711)


In [9]:
p = Pool(cpu_count()-1)
with p:
    results_queries = list(tqdm(p.imap_unordered(calculate_tamper_score, all_queries), total=len(all_queries)))

100%|████████████████████████████████████| 24000/24000 [01:05<00:00, 367.54it/s]


In [10]:
df = pd.DataFrame(columns=['type', 'len', 'bitrate', 'ref', 'query_no', 'score'], data=results_queries)

In [15]:
outdir = 'baseline'
os.makedirs(f'./daps-mp3/results/{outdir}', exist_ok=True)
df.to_csv(f'./daps-mp3/results/{outdir}/baseline_test.csv')

In [4]:
outdir = 'baseline'
df = pd.read_csv(f'./daps-mp3/results/{outdir}/baseline_test.csv', index_col=0)

In [5]:
for bitrate in ['256k', '128k', '64k']:
    get_eer_table(df, '256k')

,tamper_len,INS,DEL,REP,aggregate
0,4,0.00,0.4,0.00,0.133333
1,2,0.20,1.6,1.20,1.000000
2,1,1.60,2.0,12.00,6.200000
3,0.5,4.60,2.2,29.40,14.800000
4,0.25,5.40,3.2,39.80,19.800000
5,aggregate,2.72,2.0,19.56,9.173333


,tamper_len,INS,DEL,REP,aggregate
0,4,0.00,0.4,0.00,0.133333
1,2,0.20,1.6,1.20,1.000000
2,1,1.60,2.0,12.00,6.200000
3,0.5,4.60,2.2,29.40,14.800000
4,0.25,5.40,3.2,39.80,19.800000
5,aggregate,2.72,2.0,19.56,9.173333


,tamper_len,INS,DEL,REP,aggregate
0,4,0.00,0.4,0.00,0.133333
1,2,0.20,1.6,1.20,1.000000
2,1,1.60,2.0,12.00,6.200000
3,0.5,4.60,2.2,29.40,14.800000
4,0.25,5.40,3.2,39.80,19.800000
5,aggregate,2.72,2.0,19.56,9.173333
